# Introduction

Introduction to Synthesis Imaging: Measurement Equation, Iterative $\chi^2$
minimization, Major and Minor cycles, Algorithm Options, etc.

This section describes the framework used by CASA for interferometric
image reconstruction



## Concept :

Image reconstruction in radio interferometry is the process of solving
the linear system of equations $\vec{V} = [A] \vec{I}$, where $\vec{V}$
represents visibilities calibrated for direction independent effects,
$\vec{I}$
is a list of parameters that model the sky brightness distribution (for
example, a image of pixels) and $[A]$ is the measurement operator that
encodes the process of how visibilities are generated when a telescope
observes a sky brightness $\vec{I}$.&nbsp; $[A]$ is
generally given by $[S_{dd}][F]$ where $[F]$ represents
a 2D Fourier transform, and $[S_{dd}]$ represents a 2D spatial
frequency sampling function that can include direction-dependent
instrumental effects. For a practical interferometer with a finite
number of array elements, $[A]$ is non-invertible because of
unsampled regions of the $uv$ plane. Therefore, this system of
equations must be solved iteratively, applying constraints via various
choices of image parameterizations and instrumental models.

&nbsp;



## Implementation ( major and minor cycles ):

Image reconstruction in CASA comprises an outer loop of *major cycles*
and an inner loop of *minor cycles*. The major cycle implements
transforms between the data and image spaces and the minor cycle
operates purely in the image domain. Together, they implement an
iterative weighted $\chi^2$ minimization process that solves
the measurement equation.

&nbsp;

![26ad14d4f63ff633dbd5d9e92d40a5059ab46a67.png](26ad14d4f63ff633dbd5d9e92d40a5059ab46a67.png)

|         |                                                         |
|:--------|---------------------------------------------------------|
| Type    | Figure                                                  |
| ID      | imagingoverview-fig-majorminorcycles                    |
| Caption | Iterative Image Reconstruction - Major and Minor Cycles |

&nbsp;

The data to image transform is called the *imaging* step in which a
pseudo inverse of $[S_{dd}][F]$ is computed and applied to
the visibilities. Operationally, weighted visibilities are
convolutionally resampled onto a grid of spatial-frequency cells,
inverse Fourier transformed, and normalized. This step is equivalent to
calculating the normal equations as part of a least squares solution.
The image to data transform is called the *prediction* step and it
evaluates the measurement equation to convert a model of the sky
brightness into a list of model visibilities that are later subtracted
from the data to form residual visibilities. For both transforms,
direction dependent instrumental effects can be accounted for via
carefully constructed convolution functions.

Iterations begin with an initial guess for the image model.&nbsp; Each
major cycle consists of the prediction of model visibilities, the
calculation of residual visibilities and the construction of a residual
image. This residual image contains the effect of incomplete sampling of
the spatial-frequency plane but is otherwise normalized to the correct
sky flux units. In its simplest form, it can be written as a convolution
of the true sky image with a point spread function. The job of the minor
cycle is to iteratively build up a model of the true sky by separating
it from the point spread function. This step is also called
*deconvolution* and is equivalent to the process of solving the normal
equations as part of a least squares solution. Different reconstruction
algorithms can operate as minor cycle iterations, allowing for
flexibility in (for example) how the sky brightness is parameterized.
The imaging step can be approximate in that several direction dependent
effects, especially baseline, frequency or time-dependent ones can
sometimes&nbsp; ignored, minor cycles can be approximate in that they
use only PSF patches and do not try to be accurate over the entire
image, but the prediction step of the major cycle must be as accurate as
possible such that model components are converted to visibilities by
including all possible instrumental effects.

**Basic Sequence of Imaging Logic:**

Data : Calibrated visibilities, data weights, UV sampling function  
Input : Algorithm and iteration controls (stopping threshold, loop
gain,...)  
Output : Model Image, Restored Image, Residual Image,...  
  
Initialize the model image  
Compute the point spread function  
Compute the initial residual image  
While ( not reached global stopping criterion
)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
/\* Major Cycle \*/  
{  
&nbsp;&nbsp;&nbsp; While ( not reached minor-cycle stopping criterion
)&nbsp;&nbsp;&nbsp; /\* Minor Cycle \*/  
&nbsp;&nbsp;&nbsp; {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Find the parameters of a new
flux component  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Update the model and residual
images  
&nbsp;&nbsp;&nbsp; }  
&nbsp;&nbsp;&nbsp; Use current model image to predict model
visibilities  
&nbsp;&nbsp;&nbsp; Calculate residual visibilities (data - model)  
&nbsp;&nbsp;&nbsp; Compute a new residual image from residual
visibilities  
}  
Convolve the final model image with the fitted beam and add to the
residual image



## &nbsp;



## Algorithmic Options :

Within the CASA implementation, numerous choices are provided to enable
the user to fine-tune the details of their image reconstruction. Images
can be constructed as spectral cubes with multiple frequency channels or
single-plane wideband continuum images. One or more sub images may be
defined to cover a wide field of view without incurring the
computational expense of very large images. The iterative framework
described above is based on the Cotton-Schwab Clean algorithm
<a href="#cit3" id="ref-cit3" class="ref-cit">[3]</a>, but variants like
Hogbom Clean <a href="#cit1" id="ref-cit1" class="ref-cit">[1]</a> and
Clark Clean <a href="#cit2" id="ref-cit2" class="ref-cit">[2]</a> are
available as subsets of this framework. The major cycle allows controls
over different data weighting schemes
<a href="#cit10" id="ref-cit10" class="ref-cit">[10]</a> and convolution
functions that account for wide-field direction-dependent effects during
imaging and prediction
\[<a href="#cit6" id="ref-cit6" class="ref-cit">[6]</a>,
<a href="#cit7" id="ref-cit7" class="ref-cit">[7]</a> ,
<a href="#cit8" id="ref-cit8" class="ref-cit">[8]</a>\]. Deconvolution
options include the use of point source vs multi-scale image models
<a href="#cit4" id="ref-cit4" class="ref-cit">[4]</a> , narrow-band or
wide-band models <a href="#cit5" id="ref-cit5" class="ref-cit">[5]</a>,
controls on iteration step size and stopping criteria, and external
constraints such as interactive and non-interactive image masks. Mosaics
may be made with data from multiple pointings, either with each pointing
imaged and deconvolved separately before being combined in a final step,
or via a joint imaging and deconvolution
<a href="#cit9" id="ref-cit9" class="ref-cit">[9]</a>. Options to
combine single dish and interferometer data during imaging also exist.
More details about these algorithms can be obtained from
\[<a href="#cit10" id="ref-cit10" class="ref-cit">[10]</a>,
<a href="#cit11" id="ref-cit11" class="ref-cit">[11]</a>,
<a href="#cit12" id="ref-cit12" class="ref-cit">[12]</a>,
<a href="#cit13" id="ref-cit13" class="ref-cit">[13]</a>\]&nbsp;

&nbsp;

&nbsp;



## References :

|                 |                                                                                 |
|:----------------|---------------------------------------------------------------------------------|
| Citation Number | 1                                                                               |
| Citation Text   | J. A. Hogbom 1974 ([ADS](http://adsabs.harvard.edu/full/1974A%26AS...15..417H)) |

|                 |                                                                               |
|:----------------|-------------------------------------------------------------------------------|
| Citation Number | 2                                                                             |
| Citation Text   | B. G. Clark 1980 ([ADS](http://adsabs.harvard.edu/abs/1980A%26A....89..377C)) |

|                 |                                                                               |
|:----------------|-------------------------------------------------------------------------------|
| Citation Number | 3                                                                             |
| Citation Text   | F. R. Schwab, 1984 ([ADS](http://adsabs.harvard.edu/abs/1984AJ.....89.1076S)) |

|                 |                                                                             |
|:----------------|-----------------------------------------------------------------------------|
| Citation Number | 4                                                                           |
| Citation Text   | T. J. Cornwell, 2008 ([IEEE](http://ieeexplore.ieee.org/document/4703304/)) |

|                 |                                                                                                                      |
|:----------------|----------------------------------------------------------------------------------------------------------------------|
| Citation Number | 5                                                                                                                    |
| Citation Text   | U.Rau, 2011 ([Astronomy and Astrophysics)](https://www.aanda.org/articles/aa/abs/2011/08/aa17104-11/aa17104-11.html) |

|                 |                                                                             |
|:----------------|-----------------------------------------------------------------------------|
| Citation Number | 6                                                                           |
| Citation Text   | T. J. Cornwell, 2008 ([IEEE](http://ieeexplore.ieee.org/document/4703511/)) |

|                 |                                                                               |
|:----------------|-------------------------------------------------------------------------------|
| Citation Number | 7                                                                             |
| Citation Text   | S. Bhatnagar, 2008 ([ADS](http://adsabs.harvard.edu/abs/2008A&A...487..419B)) |

|                 |                                                                              |
|:----------------|------------------------------------------------------------------------------|
| Citation Number | 8                                                                            |
| Citation Text   | S.Bhatnagar, 2013 ([ADS](http://adsabs.harvard.edu/abs/2013ApJ...770...91B)) |

|                 |                                                                                   |
|:----------------|-----------------------------------------------------------------------------------|
| Citation Number | 9                                                                                 |
| Citation Text   | T. J. Cornwell, 1988 ([ADS](http://adsabs.harvard.edu/abs/1988A%26A...202..316C)) |

|                 |                                                                                                     |
|:----------------|-----------------------------------------------------------------------------------------------------|
| Citation Number | 10                                                                                                  |
| Citation Text   | Briggs D.S. 1999 ([Astron. Soc. Pac. Conf. Ser.](http://www.aspbooks.org/publications/180/127.pdf)) |

|                 |                                                                                                       |
|:----------------|-------------------------------------------------------------------------------------------------------|
| Citation Number | 11                                                                                                    |
| Citation Text   | Cornwell, T.J 1999 ([Astron. Soc. Pac. Conf. Ser.](http://www.aspbooks.org/publications/180/151.pdf)) |

|                 |                                                                                            |
|:----------------|--------------------------------------------------------------------------------------------|
| Citation Number | 12                                                                                         |
| Citation Text   | Cornwell, T.J., "The Generic Interferometer: II Image Solvers'', Aips++ note 184. Aug 1995 |

|                 |                                                                    |
|:----------------|--------------------------------------------------------------------|
| Citation Number | 13                                                                 |
| Citation Text   | U.Rau, 2009 ([IEEE](http://ieeexplore.ieee.org/document/5109712/)) |

&nbsp;

Bibliography

  
<sup>1.\ J.\ A.\ Hogbom\ 1974\ ([ADS](http://adsabs.harvard.edu/full/1974A%26AS...15..417H))\ [&larrhk;](#ref-cit1 "Jump back to citation 1 in the text.")</sup>

  
<sup>2.\ B.\ G.\ Clark\ 1980\ ([ADS](http://adsabs.harvard.edu/abs/1980A%26A....89..377C))\ [&larrhk;](#ref-cit2 "Jump back to citation 2 in the text.")</sup>

  
<sup>3.\ F.\ R.\ Schwab,\ 1984\ ([ADS](http://adsabs.harvard.edu/abs/1984AJ.....89.1076S))\ [&larrhk;](#ref-cit3 "Jump back to citation 3 in the text.")</sup>

  
<sup>4.\ T.\ J.\ Cornwell,\ 2008\ ([IEEE](http://ieeexplore.ieee.org/document/4703304/))\ [&larrhk;](#ref-cit4 "Jump back to citation 4 in the text.")</sup>

  
<sup>5.\ U.Rau,\ 2011\ ([Astronomy\ and\ Astrophysics)](https://www.aanda.org/articles/aa/abs/2011/08/aa17104-11/aa17104-11.html)\ [&larrhk;](#ref-cit5 "Jump back to citation 5 in the text.")</sup>

  
<sup>6.\ T.\ J.\ Cornwell,\ 2008\ ([IEEE](http://ieeexplore.ieee.org/document/4703511/))\ [&larrhk;](#ref-cit6 "Jump back to citation 6 in the text.")</sup>

  
<sup>7.\ S.\ Bhatnagar,\ 2008\ ([ADS](http://adsabs.harvard.edu/abs/2008A&A...487..419B))\ [&larrhk;](#ref-cit7 "Jump back to citation 7 in the text.")</sup>

  
<sup>8.\ S.Bhatnagar,\ 2013\ ([ADS](http://adsabs.harvard.edu/abs/2013ApJ...770...91B))\ [&larrhk;](#ref-cit8 "Jump back to citation 8 in the text.")</sup>

  
<sup>9.\ T.\ J.\ Cornwell,\ 1988\ ([ADS](http://adsabs.harvard.edu/abs/1988A%26A...202..316C))\ [&larrhk;](#ref-cit9 "Jump back to citation 9 in the text.")</sup>

  
<sup>10.\ Briggs\ D.S.\ 1999\ ([Astron.\ Soc.\ Pac.\ Conf.\ Ser.](http://www.aspbooks.org/publications/180/127.pdf))\ [&larrhk;](#ref-cit10 "Jump back to citation 10 in the text.")</sup>

  
<sup>11.\ Cornwell,\ T.J\ 1999\ ([Astron.\ Soc.\ Pac.\ Conf.\ Ser.](http://www.aspbooks.org/publications/180/151.pdf))\ [&larrhk;](#ref-cit11 "Jump back to citation 11 in the text.")</sup>

  
<sup>12.\ Cornwell,\ T.J.,\ "The\ Generic\ Interferometer:\ II\ Image\ Solvers'',\ Aips++\ note\ 184.\ Aug\ 1995\ [&larrhk;](#ref-cit12 "Jump back to citation 12 in the text.")</sup>

  
<sup>13.\ U.Rau,\ 2009\ ([IEEE](http://ieeexplore.ieee.org/document/5109712/))\ [&larrhk;](#ref-cit13 "Jump back to citation 13 in the text.")</sup>